# Split Dataset into Intput and Output Data
**1. Data Preprocessing Steps :** 
    <br>
    * Get rid of sequences with 'X', 'U', and 'B'.
    * Get rid of Outliers (3 stdev = 0.2% from left and right)
    * Zero Padd to Maximum Value 
**2. Retrieving Input and Output Data**

In [1]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
# Grab all data from parquet file
dataset = pd.read_parquet('Metal_all_20180116.snappy.parquet')

# fix random seed for reproducibility
np.random.seed(7)

### 1. Get rid of Sequence containing 'X' and 'U'.

In [3]:
# Boolean Column to indicate which row contains 'X' and 'U'.
xub_flag = list()
seq_len = list()   # This will be used later to get rid of outliers
for row in dataset.itertuples() :
    if ('X' in row[5]) or ('U' in row[5]) or ('B' in row[5]) : xub_flag.append(1)
    else : xub_flag.append(0)
    seq_len.append( len(row[5]) )

# Append the columns
dataset['XUBflag'] = xub_flag
dataset['seqLength'] = seq_len

# Exclude sequences containing 'X' and 'U' and 'B'
dataset = dataset[dataset.XUBflag != 1]
dataset = dataset.drop('XUBflag', axis=1)
dataset

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100,seqLength
0,1A0O.A,MN,"[11, 55, 57]","[13, 57, 59]",ADKELKFLVVDDFSTMRRIVRNLLKELGFNNVEEAEDGVDALNKLQ...,1,115.0,386.0,404.0,371.0,325.0,313.0,1621.0,128
1,1A25.A,CA,"[107, 110, 111, 113]","[248, 251, 252, 254]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,2,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0,149
2,1A25.A,CA,"[45, 46, 105, 107, 113]","[186, 187, 246, 248, 254]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,2,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0,149
3,1A25.A,CA,"[46, 52, 105, 106, 107]","[187, 193, 246, 247, 248]",GSPGISGGGGGILDSMERRGRIYIQAHIDREVLIVVVRDAKNLVPM...,1,632.0,5212.0,24455.0,28660.0,32138.0,33630.0,44596.0,149
4,1AJD.B,MG,"[50, 154, 321]","[51, 155, 322]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
5,1AJD.B,ZN,"[50, 101, 368, 369]","[51, 102, 369, 370]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
6,1AJD.B,ZN,"[326, 330, 411]","[327, 331, 412]",TPEMPVLENRAAQGDITAPGGARRLTGDQTAALRDSLSDKPAKNII...,1,584.0,592.0,555.0,515.0,485.0,444.0,8044.0,449
7,1ALN.A,ZN,"[101, 128, 131]","[102, 129, 132]",MHPRFQTAFAQLADNLQSALEPILADKYFPALLTGEQVSSLKSATG...,1,4526.0,4989.0,13948.0,15950.0,17149.0,17499.0,19782.0,294
8,1AS6.C,CU,"[97, 138, 147, 152]","[95, 136, 145, 150]",QGAVRKATAAEIAALPRQKVELVDPPFVHAHSQVAEGGPKVVEFTM...,1,181.0,198.0,182.0,158.0,305.0,292.0,548.0,343
9,1AVM.B,FE,"[26, 74, 160, 164]","[27, 75, 161, 165]",AVYTLPELPYDYSALEPYISGEIMELHHDKHHKAYVDGANTALDKL...,1,131.0,129.0,145.0,5796.0,5939.0,5934.0,5230.0,201


### 2. Get rid of outliers based on Sequence Length

In [4]:
# Sort by 'seqLength'
dataset = dataset.sort_values( by=['seqLength'], ascending=False )

# Get standard deviation and mean of sequence length.
seq_len = np.array(seq_len)
stdev = np.std(seq_len, dtype=np.float32)
mean = np.mean(seq_len, dtype=np.float32)
print('Standard deviation : ', stdev)
print('Mean : ', mean)

Standard deviation :  768.7711
Mean :  584.2073


In [5]:
dataset

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100,seqLength
36306,5XOG.A,MG,"[481, 483, 485]","[482, 484, 486]",MSQFPYSSAPLRSVKEVQFGLLSPEEIRAISVVKIEYPEIMDESRQ...,2,407.0,407.0,382.0,365.0,10027.0,10153.0,10108.0,1743
36308,5XOG.A,ZN,"[66, 69, 76, 79]","[67, 70, 77, 80]",MSQFPYSSAPLRSVKEVQFGLLSPEEIRAISVVKIEYPEIMDESRQ...,1,407.0,407.0,382.0,365.0,10027.0,10153.0,10108.0,1743
36307,5XOG.A,ZN,"[106, 109, 147, 167]","[107, 110, 148, 168]",MSQFPYSSAPLRSVKEVQFGLLSPEEIRAISVVKIEYPEIMDESRQ...,1,407.0,407.0,382.0,365.0,10027.0,10153.0,10108.0,1743
69887,1TWF.A,ZN,"[66, 69, 76, 79]","[67, 70, 77, 80]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
34933,2NVQ.A,MG,"[480, 482, 484]","[481, 483, 485]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
34934,2NVQ.A,ZN,"[66, 69, 76, 79]","[67, 70, 77, 80]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
64527,1K83.A,ZN,"[106, 109, 147, 166]","[107, 110, 148, 167]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
11884,3RZO.A,ZN,"[66, 69, 76, 79]","[67, 70, 77, 80]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
11885,3RZO.A,ZN,"[106, 109, 147, 166]","[107, 110, 148, 167]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733
7975,3S14.A,ZN,"[106, 109, 147, 166]","[107, 110, 148, 167]",MVGQQYSSAPLRTVKEVQFGLFSPEEVRAISVAKIRFPETMDETQT...,1,407.0,407.0,382.0,365.0,340.0,323.0,234.0,1733


In [6]:
# 3 standard deviations from left and right side (0.2% from each side)
std3 = int(len(dataset) * 0.002)
dataset = dataset[:-std3]
dataset = dataset[std3:]
dataset

,structureChainId,ligandId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100,seqLength
10603,2BE5.N,ZN,"[1111, 1188, 1193, 1200, 1203]","[1112, 1189, 1194, 1201, 1204]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
10602,2BE5.N,MG,"[738, 740, 742]","[739, 741, 743]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
4585,3EQL.D,MG,"[738, 740, 742]","[739, 741, 743]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
57123,3EQL.N,ZN,"[1111, 1193, 1200, 1203]","[1112, 1194, 1201, 1204]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
4586,3EQL.D,ZN,"[1111, 1193, 1200, 1203]","[1112, 1194, 1201, 1204]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
4587,3EQL.D,ZN,"[57, 59, 72, 75]","[58, 60, 73, 76]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
23151,5VOI.D,MG,"[738, 740, 742]","[739, 741, 743]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
31740,2A6H.N,ZN,"[57, 59, 72, 75]","[58, 60, 73, 76]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
62332,2A6E.D,ZN,"[1111, 1193, 1200, 1203]","[1112, 1194, 1201, 1204]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524
62331,2A6E.D,MG,"[738, 740, 742]","[739, 741, 743]",MKKEVRKVRIALASPEKIRSWSYGEVEKPETINYRTLKPERDGLFD...,1,597.0,609.0,571.0,538.0,501.0,625.0,450.0,1524


### 3. Zero Padd to the Maximum Length

In [7]:
maxlength = dataset.iloc[0, len(dataset.columns)-1]
maxlength

1524

In [8]:
# Zero padd to the max length
for row in dataset.itertuples() :
    if not row[len(dataset.columns)] == maxlength : 
        dataset.loc[row.Index, 'sequence'] = row[5] + ('0'*(maxlength-row[len(dataset.columns)]))

### 3.1 Split into Training Set and Test Set (67% : 33%)

In [9]:
train_num = int(len(dataset) * 0.67)
test_num = len(dataset) - train_num

# Splitting Input Data from the Dataframe
x_train = dataset[:train_num]
x_test = dataset[train_num:]

### 4. Get Input Data

In [10]:
# Dictionary of Amino Acids
AADICT = {'A' : 0, 'C' : 1, 'D' : 2, 'E' : 3, 'F' : 4, 'G' : 5, 'H' : 6, 'I' : 7, 'K' : 8, \
          'L' : 9, 'M' : 10, 'N' : 11, 'P' : 12, 'Q' : 13, 'R' : 14, 'S' : 15, 'T' : 16, 'V' : 17, \
          'W' : 18, 'Y' : 19}

def one_hot_encode( seq ) : 
    mat = list()
    for aa in seq :
        vec = [0] * 20
        if not aa == '0' :
            vec[ AADICT[aa] ] = 1
        mat.append(vec) 
    return np.array(mat).transpose()

### 5. Save Input Data as One-Hot ( 20 x len(seq) )

In [ ]:
indata = list()
indata_test = list()

for row in x_train.itertuples() :
    indata.append( one_hot_encode(row[5]) )

for row in x_test.itertuples() :
    indata_test.append( one_hot_encode(row[5]) )

### 6. Save Output Data as 1 or 0 ( Binary Classification )

In [ ]:
outdata = list()
outdata_test = list()

for row in x_train.itertuples() :
    if row[2] == 'ZN' : outdata.append(1)
    else : outdata.append(0)

for row in x_test.itertuples() :
    if row[2] == 'ZN' : outdata_test.append(1)
    else : outdata_test.append(0)

# Build CNN to Classify Zinc Binding Sequences
* **Sequential** - Initializes a neural network as a sequence of layers.
* **Convolution2D** - Creates convolutional layers. (images = 2D, videos = 3D)
* **MaxPooling2D** - Creates pooling layers.
* **Flatten** - Convertes pooled feature maps into a feature vector.
* **Dense** - Creates fully connected layer.

In [ ]:
import time
import matplotlib.pyplot as plt
% matplotlib inline

# Importing all the necessary libraries to build CNN
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense

We first need to initialize the model.

In [ ]:
classifier = Sequential()

#### Step 1. Convolutional Layer + ReLu
Use **Conv2D** function from **keras.layers.Conv2D**.<br>
Parameters : <br>
* **filter** - number of filters. (number of filters = number of feature maps.)
* **kernel_size** - tuple/list of 2 integers (# of rows, # of cols).
* **input_shape** - (row pixel, col pixel, 3 for RGB) pictures. (All image will probably have different size. This will allow images to all have fixed size.)
* **activation** - activation function.

In [ ]:
# This will create 32 Feature Maps.
classifier.add(Conv2D(32, (3,3), input_shape=(20, maxlength, 1), activation="relu"))

#### Step 2. Max Pooling

In [ ]:
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

We can make this model **deeper** in order to create better accuracy by creating **another convolultional layer**.

In [ ]:
# Don't need 'input_shape' because it's already given from previous step.
classifier.add(Conv2D(32, (3,3), activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

#### Step 3. Flattening

In [ ]:
classifier.add(Flatten())

#### Step 4. Fully Connected Layer
Use **Dense** to create fully connected layer.
* **units** - # nodes in output layer ( basically hidden layer because hidden is the output layer from input layer ).
* **activation** - activation function

In [ ]:
# Choose units to be not too big but not too small. Common practice is to pick power of 2.
classifier.add(Dense(units=128, activation="relu"))

In [ ]:
# Hidden layer to Output layer
classifier.add(Dense(units=1, activation="sigmoid"))

#### Step 4.1 Compile CNN
This is to configure the learning process. 

In [ ]:
# Binary_crossentropy because we are classifying 2 outcomes.
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
# Fit the model (TRAINING)
indata = np.array(indata).reshape(len(x_train), 20, maxlength, 1)
start = time.time()
classifier.fit(indata, outdata, validation_split=0.33, epochs=20, batch_size=10)
end = time.time()
print("Model took %0.2f seconds" %(end - start))

In [ ]:
score = model.evaluate(indata_test, outdata_test, verbose=0)